In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tensorflow_hub.module_v2 import load

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')
print(tf.__version__)

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encount

2.9.0


In [2]:
train_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/train',
    './project-6-at-2023-04-30-00-12-74270469/train',
    ['Counted_Objects', 'Not_Counted_Objects']
)

val_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/validate',
    './project-6-at-2023-04-30-00-12-74270469/validate',
    ['Counted_Objects', 'Not_Counted_Objects']
)

In [3]:
print(train_ds.size)
print(val_ds.size)

108
27


In [4]:
#spec = model_spec.get('efficientdet_lite4')

spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite4', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/2', 
  hparams={'max_instances_per_image': 8000})

In [5]:
model = object_detector.create(train_ds, model_spec=spec, batch_size=8, train_whole_model=True, epochs=120, validation_data=val_ds)

Epoch 1/120
13/13 [==============================] - 312s 20s/step - det_loss: 1.7513 - cls_loss: 1.1200 - box_loss: 0.0126 - reg_l2_loss: 0.1078 - loss: 1.8591 - learning_rate: 0.0090 - gradient_norm: 0.5997 - val_det_loss: 1.6655 - val_cls_loss: 1.0896 - val_box_loss: 0.0115 - val_reg_l2_loss: 0.1078 - val_loss: 1.7733
Epoch 2/120
13/13 [==============================] - 259s 20s/step - det_loss: 1.5979 - cls_loss: 1.0294 - box_loss: 0.0114 - reg_l2_loss: 0.1078 - loss: 1.7057 - learning_rate: 0.0100 - gradient_norm: 1.1861 - val_det_loss: 1.3641 - val_cls_loss: 0.8528 - val_box_loss: 0.0102 - val_reg_l2_loss: 0.1078 - val_loss: 1.4719
Epoch 3/120
13/13 [==============================] - 239s 19s/step - det_loss: 1.2817 - cls_loss: 0.7709 - box_loss: 0.0102 - reg_l2_loss: 0.1078 - loss: 1.3895 - learning_rate: 0.0100 - gradient_norm: 2.6935 - val_det_loss: 1.0670 - val_cls_loss: 0.6001 - val_box_loss: 0.0093 - val_reg_l2_loss: 0.1078 - val_loss: 1.1749
Epoch 4/120
13/13 [============

In [6]:
model.evaluate(train_ds)

2/2 [==============================] - 60s 21s/step



{'AP': 0.6018948,
 'AP50': 0.80264163,
 'AP75': 0.7107133,
 'APs': 0.0,
 'APm': 0.0029702971,
 'APl': 0.6161316,
 'ARmax1': 0.178109,
 'ARmax10': 0.46380356,
 'ARmax100': 0.665302,
 'ARs': 0.0,
 'ARm': 0.0031578948,
 'ARl': 0.68124384,
 'AP_/Counted_Objects': 0.5839277,
 'AP_/Not_Counted_Objects': 0.6198619}

In [7]:
model.evaluate(val_ds)

1/1 [==============================] - 17s 17s/step



{'AP': 0.4288148,
 'AP50': 0.6325704,
 'AP75': 0.5119319,
 'APs': 0.0,
 'APm': 0.090841584,
 'APl': 0.43219337,
 'ARmax1': 0.1379662,
 'ARmax10': 0.3470587,
 'ARmax100': 0.5164886,
 'ARs': 0.0,
 'ARm': 0.10909091,
 'ARl': 0.5200362,
 'AP_/Counted_Objects': 0.41595903,
 'AP_/Not_Counted_Objects': 0.44167057}

In [8]:
model.export(export_dir='../obj_model_training/',tflite_filename='efd_4_uc_ma_bs8.tflite')